In [1]:
access_token = "760244356783538176-nCtOfjvowhbmDqj45hN9EOwSXLWDd7h"
access_token_secret = "m40tqPuHhJG5Ti6TIB3DbhZqVPMrouFsByksjUbSFUbZW"
comsumer_key = "WwbD7n34mOHHepH92LLJm2W9Z"
consumer_secret = "TY2flAOk1bgPS9Z7R8HJP9pYXDOSuMEk4DhPJi4k4ZMLLqm47G"

In [2]:
from pymongo import MongoClient
def get_mongo_database(db_name, host='localhost', port=27017, username=None, password=None):
    if username and password:
        mongo_uri = 'mongodb://{}:{}@{}/()'.format(username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    
    return conn[db_name]

In [3]:
import json
import tweepy
from tweepy.streaming import StreamListener

class MyStreamListener(StreamListener):
    def __init__(self, api, **kw):
        self.api = api
        super(StreamListener, self).__init__()
        self.col = get_mongo_database('tweets', **kw)['tweets']
    
    def on_data(self, tweet):
        self.col.insert_one(json.loads(tweet))
        
    def on_error(self, status):
        return True  # keep stream open

In [4]:
auth = tweepy.OAuthHandler(comsumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [5]:
stream = tweepy.Stream(auth, MyStreamListener(api))

In [6]:
stream.filter(track=['python', 'javascript', 'deep learning', 'AI'])

KeyboardInterrupt: 